In [1]:
import pandas as pd
import imblearn
import numpy as np
import traceback
from dateutil.parser import parse
pd.set_option('display.max_rows', 20,'max_info_columns', 9999,'display.max_columns', 9999)
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing,metrics
import datetime as dt
import gc
from sklearn import svm,linear_model
from collections import Counter
from sklearn.model_selection import train_test_split
import csv
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder,Imputer

from sklearn.ensemble import RandomForestRegressor,VotingClassifier
from sklearn.kernel_ridge import KernelRidge
from sklearn import neighbors
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedKFold


from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cross_validation import  cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
import xlrd
warnings.filterwarnings('ignore')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_table('/home/tsl-yu/文档/天池-盐城-上牌量预测/data/train_20171215.txt')
test=pd.read_table('/home/tsl-yu/文档/天池-盐城-上牌量预测/data/test_A_20171225.txt')
sample=pd.read_table('/home/tsl-yu/文档/天池-盐城-上牌量预测/data/sample_A_20171225.txt',names='s')

In [3]:
train = train.groupby(['date','day_of_week'], as_index=False)['cnt'].agg({'label':np.sum})

In [4]:
label=train['label']
train.drop('label',axis=1,inplace=True)

In [5]:
train=pd.concat([train,test],axis=0)

In [6]:
sample.reset_index(inplace=True)

##  交叉验证函数

In [7]:
def cv_function(model,train,label,test,n=10):
    try:
        kf = KFold(n_splits=n,shuffle=True)
        l=[]
        
        err=[]
        if type(train)==pd.DataFrame:
            train=train.fillna(-999).values
        if type(test)==pd.DataFrame:
            test=test.fillna(-999).values
        
        if type(label)==pd.core.series.Series:
            label=label.values
    
        for tr_index,te in kf.split(train,label):
            train_x,test_x=train[tr_index],train[te]
            train_y,test_y=label[tr_index],label[te]
            model.fit(train_x,train_y)
            a=model.predict(test_x)
            err.append(np.mean((a-test_y)**2))
            print (err[-1])
            
            out=model.predict(test)
            l.append(out)
                
            
        print('mean error   ',np.mean(err))
        return l,err
    except :
        print('error')

## 特征工程

In [ ]:
train['day']=train['date']%30                     #日期特征

In [ ]:
train['month']=np.ceil(train['date']/30)%12   
train['month']=train['month'].replace({ 0.:12})

In [ ]:
train['season']=train['month']%4
train['season']=train['season'].replace({0:4})

In [ ]:
train['year']=np.ceil(train['date']/344)

In [ ]:
train.drop('date',axis=1,inplace=True)

##  xgb 模型训练

In [38]:
train_x,test_x,train_y,test_y=train_test_split(train.iloc[:1032],label,test_size=0.2,random_state=10)

train_=xgb.DMatrix(train_x,train_y)
test_=xgb.DMatrix(test_x,test_y)

watchlist=[(train_,'train'),(test_,'test')]

In [43]:
    params = {
            'booster' : 'gbtree',
            'objective': 'reg:linear',
            'eta': 0.02,
            'gamma' : 1,
            'max_depth' :3,
            'min_child_weight' :1,
            'eval_metric':'rmse',
            'seed': 0,
            'missing': -999,
            'subsample' :0.8,
            'alpha'   :  1,
             'lambda'  :   10  ,
            'colsample_bytree' :0.8,
             'colsample_bybooster':1,
          
            'silent' : 1,
            
            
            }

In [44]:
model = xgb.train(params,train_,num_boost_round=500 ,evals=watchlist,early_stopping_rounds=30)

[0]	train-rmse:2093.96	test-rmse:1994.16
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 30 rounds.
[1]	train-rmse:2064.54	test-rmse:1965.81
[2]	train-rmse:2032.78	test-rmse:1935.48
[3]	train-rmse:2005.31	test-rmse:1909.19
[4]	train-rmse:1978.89	test-rmse:1883.77
[5]	train-rmse:1952.7	test-rmse:1858.67
[6]	train-rmse:1927.17	test-rmse:1834.14
[7]	train-rmse:1902.47	test-rmse:1810.46
[8]	train-rmse:1878.29	test-rmse:1787.41
[9]	train-rmse:1849.83	test-rmse:1760.44
[10]	train-rmse:1823.14	test-rmse:1735.13
[11]	train-rmse:1796.78	test-rmse:1710.35
[12]	train-rmse:1775.5	test-rmse:1690
[13]	train-rmse:1754.59	test-rmse:1670.38
[14]	train-rmse:1729.92	test-rmse:1647.08
[15]	train-rmse:1705.64	test-rmse:1624.47
[16]	train-rmse:1682.25	test-rmse:1602.43
[17]	train-rmse:1659.18	test-rmse:1580.97
[18]	train-rmse:1639.69	test-rmse:1562.9
[19]	train-rmse:1617.56	test-rmse:1542.11
[20]	train-rmse:1599.63	test-rmse

[191]	train-rmse:776.955	test-rmse:872.15
[192]	train-rmse:775.922	test-rmse:871.738
[193]	train-rmse:774.969	test-rmse:871.453
[194]	train-rmse:773.719	test-rmse:870.928
[195]	train-rmse:773.156	test-rmse:870.946
[196]	train-rmse:772.773	test-rmse:870.716
[197]	train-rmse:772.267	test-rmse:870.682
[198]	train-rmse:771.554	test-rmse:870.711
[199]	train-rmse:771.209	test-rmse:870.569
[200]	train-rmse:770.825	test-rmse:870.353
[201]	train-rmse:769.631	test-rmse:870.034
[202]	train-rmse:769.345	test-rmse:869.831
[203]	train-rmse:768.99	test-rmse:869.521
[204]	train-rmse:768.682	test-rmse:869.389
[205]	train-rmse:768.42	test-rmse:869.199
[206]	train-rmse:767.824	test-rmse:869.198
[207]	train-rmse:766.998	test-rmse:869.025
[208]	train-rmse:766.166	test-rmse:868.64
[209]	train-rmse:765.95	test-rmse:868.595
[210]	train-rmse:765.27	test-rmse:868.635
[211]	train-rmse:764.801	test-rmse:868.666
[212]	train-rmse:764.516	test-rmse:868.603
[213]	train-rmse:763.387	test-rmse:868.319
[214]	train-rmse:

[383]	train-rmse:713.463	test-rmse:858.754
[384]	train-rmse:713.237	test-rmse:858.613
[385]	train-rmse:712.758	test-rmse:858.275
[386]	train-rmse:712.519	test-rmse:858.414
[387]	train-rmse:712.498	test-rmse:858.48
[388]	train-rmse:712.491	test-rmse:858.602
[389]	train-rmse:712.075	test-rmse:858.645
[390]	train-rmse:711.917	test-rmse:858.65
[391]	train-rmse:711.913	test-rmse:858.631
[392]	train-rmse:711.906	test-rmse:858.576
[393]	train-rmse:711.489	test-rmse:858.617
[394]	train-rmse:711.482	test-rmse:858.367
[395]	train-rmse:711.266	test-rmse:858.52
[396]	train-rmse:710.992	test-rmse:858.664
[397]	train-rmse:710.98	test-rmse:858.504
[398]	train-rmse:710.452	test-rmse:858.299
[399]	train-rmse:710.442	test-rmse:858.287
[400]	train-rmse:710.377	test-rmse:858.165
[401]	train-rmse:709.83	test-rmse:858.299
[402]	train-rmse:709.826	test-rmse:858.321
[403]	train-rmse:709.816	test-rmse:858.39
[404]	train-rmse:709.258	test-rmse:858.522
[405]	train-rmse:709.252	test-rmse:858.232
[406]	train-rmse:

In [31]:
score=model.get_score()

impt=pd.DataFrame()
impt['name']=score.keys()
impt['importance']=score.values()

impt=impt.sort_values(by ='importance',axis=0,ascending=False)
impt.reset_index(drop=True,inplace=True)

In [ ]:
impt

##  交叉验证

In [86]:
xrg=xgb.XGBRegressor(max_depth=3,learning_rate=0.02,
n_estimators=400,nthread=-1,gamma=0.1,colsample_bytree=0.81,
        subsample=0.8,reg_alpha=0.1, reg_lambda=1,
missing=-999,colsample_bylevel=1)

In [100]:
k,err=cv_function(xrg,train.iloc[:1032],label,train.iloc[1032:],n=5)

880577.8787863082
485295.50616714347
380919.9074357292
489499.91081705055
744244.9326461976
mean error    596107.6271704858


In [101]:
err1=[1/a for a in err]
err1=err1/sum(err1)

out_xgb=[(k[i])*err1[i] for i in range(len(k))]

out_xgb=sum(k)/len(k)

In [102]:
sample['s']=out_xgb

In [103]:
sample.to_csv('result_only_date.txt',index=False,header=False,sep='\t')

In [ ]:
sns.distplot(label)

In [ ]:
sns.jointplot(train['date'],label,)

In [ ]:
sns.jointplot(train['date'][:344],label[:344])

In [ ]:
sns.jointplot(train['date'][344:688],label[344:688])

In [ ]:
sns.jointplot(train['date'][688:],label[688:])